# Metadata creation

In [1]:
ROOT_DIR = "../../"

In [2]:
import os
import sys
import json
import pandas as pd
from scipy.stats import zscore

sys.path.insert(0, os.path.abspath(ROOT_DIR + 'src/scraping'))
import youtube_requesting as ytr
import metadata as mdata

In [3]:
with open(ROOT_DIR + "config/" + "config-scraping.json") as f:
    cfg = json.load(f)
with open(ROOT_DIR + "api_key.json") as json_file:
    cred = json.load(json_file)

In [4]:
api_service_name = "youtube"
api_version = "v3"

In [5]:
api_key = cred['api_key']
game = cfg["selected-game"]

# data = mdata.fetch_raw_data(ROOT_DIR + cfg["videos-dir"].format(game) + '/', cfg)
# df = mdata.make_mdata_df(data["04_16_20"], api_key)
# df.to_csv(ROOT_DIR + cfg["videos-dir"].format(game) + '/{}_metadata.csv'.format(game), index=False)

In [6]:
df

NameError: name 'df' is not defined

In [9]:
eval(df.thumbnails[0])

{'default': {'url': 'https://i.ytimg.com/vi/D4sP53qTfwk/default.jpg',
  'width': 120,
  'height': 90},
 'medium': {'url': 'https://i.ytimg.com/vi/D4sP53qTfwk/mqdefault.jpg',
  'width': 320,
  'height': 180},
 'high': {'url': 'https://i.ytimg.com/vi/D4sP53qTfwk/hqdefault.jpg',
  'width': 480,
  'height': 360},
 'standard': {'url': 'https://i.ytimg.com/vi/D4sP53qTfwk/sddefault.jpg',
  'width': 640,
  'height': 480},
 'maxres': {'url': 'https://i.ytimg.com/vi/D4sP53qTfwk/maxresdefault.jpg',
  'width': 1280,
  'height': 720}}

# Downloading thumbnails

In [10]:
for qual in cfg["thumbnail-qual"]:
    if cfg["thumbnail-qual"][qual]:
        mdata.download_df_thumbs(df, ROOT_DIR + cfg["thumbnails-dir"].format(cfg["selected-game"]), res=qual)

In [11]:
df = pd.read_csv(ROOT_DIR + cfg["videos-dir"].format(game) + "{}_full_metadata.csv".format(game))

In [12]:
df.head()

,categoryId,channelId,channelTitle,commentCount,date,defaultLanguage,description,dislikeCount,duration,favoriteCount,...,tags,thumbnails,title,videoId,viewCount,z_comments,z_dislikes,z_likes,z_views,position
0,20.0,UC_q5WZtFp36adwqhKpZzxwQ,SypherPK,3055.0,2020-04-16 17:00:05+00:00,en,You guys always LOVE these 100 streamsniper vi...,1475.0,PT15M1S,0.0,...,"['fortnite', 'fortnite battle royale', 'compet...",{'default': {'url': 'https://i.ytimg.com/vi/D4...,I Told 100 Streamsnipers To Drop PRISON and WO...,D4sP53qTfwk,2354152.0,0.133964,0.882870,1.167817,0.951791,0
1,20.0,UCqsBym4OHrzSp0Nq1eZoMIA,mrfreshasian,5528.0,2020-04-14 19:39:33+00:00,NaN,Use Code: FRESH | SUBSCRIBE! https://goo.gl/gK...,672.0,PT15M38S,0.0,...,"['fortnite', 'fortnitebr', 'fortnite battle ro...",{'default': {'url': 'https://i.ytimg.com/vi/3k...,THE BEST SOLO PLAYER IN FORTNITE,3k6_ymss6NA,1876572.0,2.227322,0.840935,1.772628,1.003991,1
2,20.0,UCuMVy-cfz29-qSMxz0ZbzcQ,Yung Chip,1617.0,2020-04-15 21:53:52+00:00,en,PROTECT THE PRESIDENT challenge in Fortnite......,331.0,PT15M57S,0.0,...,"['fortnite', 'fortnite protect the president',...",{'default': {'url': 'https://i.ytimg.com/vi/yt...,PROTECT THE PRESIDENT challenge in Fortnite...,ytJuE6RMKUw,496386.0,-0.009388,-0.378903,-0.475684,-0.307306,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Uzv_799UxkM,NaN,NaN,NaN,NaN,NaN,3
4,20.0,UC-sjC5fKKsrpcZRJwlZJzZQ,McCreamy,3334.0,2020-04-14 21:16:16+00:00,NaN,this is why i hate competitive fortnite \n\nLI...,892.0,PT10M55S,0.0,...,"['fortnite', 'fortnite battle royale', 'fortni...",{'default': {'url': 'https://i.ytimg.com/vi/JZ...,i hate competitive fortnite..,JZafBWQ7lWc,1682309.0,-0.409885,-0.770639,-0.688091,-0.905637,4


# Statistics

### Desired functions:
- Given a video_id, get the surrounding videos and z-score that video in comparison.
- Get the basic video statistics surrounding a video
    - Likes
    - Dislikes
    - Views
    - Subscriber count
    - Video length(?)
- Combination of all stats to form a "success score"

### Note on api costs
Calculate those costs here:
https://developers.google.com/youtube/v3/determine_quota_cost

In [ ]:
# Parameters

# When calculating z-score of a video, grab the +- x videos surrounding the target video.
nearest_videos = 5

## Possible success metrics
Available measurements:
- date
- views (COUNT)
- likes (COUNT)
- dislikes (COUNT)
- comments (COUNT)

Metadata recommendations from here (page 6-7):
https://www.sciencedirect.com/science/article/pii/S187705091731757X/pdf?md5=5961456d3adc0f65e03cff17279317d2&pid=1-s2.0-S187705091731757X-main.pdf

Get a number of videos from the parent channel closest to the selected video in date and zscore the available measurements, then average those measurements.

Weight views higher?

Make another function that can take in an input of a list of video ids

## Statistics
#### (WIP)
Add in a measurement for video time length

Expecting a new input

`
{'video_id': '4uf2x-O_pBw',
 'position': '0',
 'channel_videos': ['4uf2x-O_pBw', '8cu4qAoeXow', 'TZK-sk5Yvfw'],
 'channel_id':str}
`

In [ ]:
def get_channel_vids_stats(videos):
    stats = []
    for v in videos:
        curr_vid = ytr.request_video_details(v,
                                             api_key,
                                             api_service_name,
                                             api_version)['items'][0]
        stats.append(ytr.get_vid_stats(curr_vid))
        time.sleep(1)
    
    return pd.DataFrame(stats)

In [ ]:
def video_success(in_dict):
    vid_stats = get_channel_vids_stats(in_dict["channel_videos"])
    
    # A lower number position means the video is more popular.
    position_weight = 1.5
    position_score = (1 - (in_dict["position"] / len(in_dict["channel_videos"]))) * position_weight
    
    zscores = vid_stats[["comments", "dislikes", "likes", "views"]].apply(scipy.stats.zscore)
    
    return position_score + zscores.loc[in_dict["position"]].sum()

In [ ]:
def video_success(row, weights=[1 for _ in len(row)]):
    position_score = (1 - (row.loc["position"] / len(in_dict["channel_videos"]))) * position_weight

In [13]:
df.keys()

Index(['categoryId', 'channelId', 'channelTitle', 'commentCount', 'date',
       'defaultLanguage', 'description', 'dislikeCount', 'duration',
       'favoriteCount', 'likeCount', 'tags', 'thumbnails', 'title', 'videoId',
       'viewCount', 'z_comments', 'z_dislikes', 'z_likes', 'z_views',
       'position'],
      dtype='object')

In [33]:
from scipy.stats import zscore

In [26]:
def global_video_success(row, weights=None):
    metric_cols = ["commentCount", "dislikeCount", "favoriteCount",
                   "likeCount", "viewCount"]
    if weights is None:
        weights = [1 for _ in metric_cols]
    weights[1] = -1
    
    scores = [row[key] for key in metric_cols]
    return sum(scores)

In [28]:
def channel_video_success(row, weights=None):
    metric_cols = ["z_comments", "z_dislikes",
                   "z_likes", "z_views"]
    if weights is None:
        weights = [1 for _ in metric_cols]
    
    scores = [row[key] for key in metric_cols]
    return sum(scores)

In [38]:
def get_success_metrics(df):
    df["global_success"] = df.apply(global_video_success, axis=1)
    df["global_success"] = zscore(df["global_success"], nan_policy="omit")
    df["channel_success"] = df.apply(channel_video_success, axis=1)
    
    return df

In [39]:
get_success_metrics(df)

,categoryId,channelId,channelTitle,commentCount,date,defaultLanguage,description,dislikeCount,duration,favoriteCount,...,title,videoId,viewCount,z_comments,z_dislikes,z_likes,z_views,position,global_success,channel_success
0,20.0,UC_q5WZtFp36adwqhKpZzxwQ,SypherPK,3055.0,2020-04-16 17:00:05+00:00,en,You guys always LOVE these 100 streamsniper vi...,1475.0,PT15M1S,0.0,...,I Told 100 Streamsnipers To Drop PRISON and WO...,D4sP53qTfwk,2354152.0,0.133964,0.882870,1.167817,0.951791,0,0.886594,3.136442
1,20.0,UCqsBym4OHrzSp0Nq1eZoMIA,mrfreshasian,5528.0,2020-04-14 19:39:33+00:00,NaN,Use Code: FRESH | SUBSCRIBE! https://goo.gl/gK...,672.0,PT15M38S,0.0,...,THE BEST SOLO PLAYER IN FORTNITE,3k6_ymss6NA,1876572.0,2.227322,0.840935,1.772628,1.003991,1,0.528917,5.844876
2,20.0,UCuMVy-cfz29-qSMxz0ZbzcQ,Yung Chip,1617.0,2020-04-15 21:53:52+00:00,en,PROTECT THE PRESIDENT challenge in Fortnite......,331.0,PT15M57S,0.0,...,PROTECT THE PRESIDENT challenge in Fortnite...,ytJuE6RMKUw,496386.0,-0.009388,-0.378903,-0.475684,-0.307306,2,-0.539413,-1.171282
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Uzv_799UxkM,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
4,20.0,UC-sjC5fKKsrpcZRJwlZJzZQ,McCreamy,3334.0,2020-04-14 21:16:16+00:00,NaN,this is why i hate competitive fortnite \n\nLI...,892.0,PT10M55S,0.0,...,i hate competitive fortnite..,JZafBWQ7lWc,1682309.0,-0.409885,-0.770639,-0.688091,-0.905637,4,0.385029,-2.774252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,20.0,UCyeVfsThIHM_mEZq7YXIQSQ,MoreAliA,1468.0,2020-03-23 18:01:00+00:00,en-GB,Top Rarest Fortnite clips you'll ever see - En...,1254.0,PT10M2S,0.0,...,The *RAREST* Storm Circle you'll EVER see in F...,J5XnEjlNEpQ,1361212.0,-0.464469,-0.184593,-0.198109,0.039175,195,0.132411,-0.807997
196,20.0,UCke6I9N4KfC968-yRcd5YRg,SSundee,18848.0,2020-03-19 11:00:02+00:00,NaN,WE do a RANDOM BOSS Skin challenge in Fornite ...,6011.0,PT15M56S,0.0,...,RANDOM Boss SKIN CHALLENGE in Fortnite ARENA,0wCfuBysnZI,7038742.0,2.322030,3.686101,3.590831,3.487868,196,4.676136,13.086830
197,20.0,UCxVaMZgCE9gB4EfMcityz6A,KingAlmightyyy,5365.0,2020-04-12 05:00:12+00:00,nl,nieuwe v-bucks coins alle geheime locaties in ...,213.0,PT11M41S,0.0,...,nieuwe v-bucks coins alle geheime locaties in ...,cEBFep4vn6o,104858.0,3.406368,-0.182626,-0.111279,-0.235872,197,-0.845722,2.876591
198,20.0,UCIwWkPe5yq_jq1_1d2bC05w,Mattiz,1196.0,2020-04-14 09:11:16+00:00,NaN,Scopriamo cosa succede se prendi tutte le chia...,694.0,PT13M34S,0.0,...,COSA SUCCEDE SE PRENDI TUTTE LE CHIAVI DEI BOS...,MsbDxAu6JLQ,451000.0,1.036510,0.846191,0.653333,1.121683,198,-0.580200,3.657718


***

In [ ]:
def get_channel_zscores(channel_id, df):
    df_subset = df[df["channelId"] == channel_id]
    target_cols = ['commentCount', 'dislikeCount',
                   'favoriteCount', 'likeCount', 'viewCount']
    
    df_zscores = df_subset[target_cols].apply(lambda col:zscore(col, nan_policy="omit"))
    df_zscores["video_id"] = df_subset["video_id"]
    return df_zscores

In [ ]:
df.keys()

In [ ]:
def get_channel_zscores(row):
    channel_vids = row.loc[""]
    target_cols = ['commentCount', 'dislikeCount',
                   'favoriteCount', 'likeCount', 'viewCount']
    
    df_zscores = df_subset[target_cols].apply(lambda col:zscore(col, nan_policy="omit"))
    df_zscores["video_id"] = df_subset["video_id"]
    return df_zscores

In [ ]:
def get_df_zscores(df):
    channel_zscores = {}
#     for channel in unique_channels:
#         channel_zscores[channel] = get_channel_zscores(channel, df, channel_zscores)
    channel_zscores = df.apply(get_channel_zscores, axis=1)
        
    return channel_zscores

In [ ]:
get_df_zscores(df)

In [ ]:
df.keys()

In [ ]:
target_cols = ['video_id', 'commentCount', 'dislikeCount',
                   'favoriteCount', 'likeCount', 'viewCount']
df[target_cols]["commentCount"].isna().sum()

In [ ]:
df